In [ ]:
using Turing, Distributions
using PyPlot, PyCall

In [ ]:
y1 = 10*sin(0:0.2:2pi); y1 += randn(length(y1))
y2 = 10*sin(0:0.2:2pi); y2 += randn(length(y2))
y = [y1'; y2']
N = size(y)[end];  K = 20;

In [ ]:
@model FHMM(y) = begin
    s1 = tzeros(Int, N)
    s2 = tzeros(Int, N)
    m1 = tzeros(Real, K)
    m2 = tzeros(Real, K)
    T1 = Vector{Vector{Real}}(K)
    T2 = Vector{Vector{Real}}(K)
    for i = 1:K
        T1[i] ~ Dirichlet(ones(K)/K)
        T2[i] ~ Dirichlet(ones(K)/K)
        m1[i] ~ Normal(i, 1)
        m2[i] ~ Normal(i, 1)
    end
    s1[1] ~ Categorical(ones(Float64, K)/K)
    s2[1] ~ Categorical(ones(Float64, K)/K)
    for i = 2:N
        s1[i] ~ Categorical(vec(T1[s1[i-1]]))
        s2[i] ~ Categorical(vec(T2[s2[i-1]]))
        y[:,i] ~ MvNormal([m1[s1[i]], m2[s2[i]]], 1*ones(2))
    end
end

In [ ]:
g = Gibbs(300, HMC(1, 0.2, 5, :m1, :T1, :m2, :T2), PG(25, 1, :s1, :s2); thin=false)
c = sample(FHMM(y), g);

In [ ]:
describe(c)

In [ ]:
m1 = c[:m1][222];
s1 = c[:s1][222];
PyPlot.plot(y[1,:], linestyle="None", marker="+", color = "r")
PyPlot.plot(m1[s1], linestyle="-", marker="*", color = "b")

In [ ]:
m2 = c[:m2][250];
s2 = c[:s2][250];
PyPlot.plot(y[2,:], linestyle="None", marker="+", color = "r")
PyPlot.plot(m2[s2], linestyle="-", marker="*", color = "b")